In [1]:
import pandas as pd
import re
from datetime import datetime
pd.set_option('display.max_colwidth', -1)

df = pd.read_csv('E:\py\codecademy\codecademy-python\jeopardy_starting\jeopardy_starting\jeopardy.csv')
print(df.info())

#title in columns has space,strip space
df.columns = df.columns.str.strip()
#print(df.columns.tolist())
print(df.head(-5))

<ipython-input-1-b65a7b2ffc76>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216930 entries, 0 to 216929
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Show Number  216930 non-null  int64 
 1    Air Date    216930 non-null  object
 2    Round       216930 non-null  object
 3    Category    216930 non-null  object
 4    Value       216930 non-null  object
 5    Question    216930 non-null  object
 6    Answer      216928 non-null  object
dtypes: int64(1), object(6)
memory usage: 11.6+ MB
None
        Show Number    Air Date             Round  \
0       4680         2004-12-31  Jeopardy!          
1       4680         2004-12-31  Jeopardy!          
2       4680         2004-12-31  Jeopardy!          
3       4680         2004-12-31  Jeopardy!          
4       4680         2004-12-31  Jeopardy!          
...      ...                ...        ...          
216920  4999         2006-05-11  Double Jeopardy!   
216921  4999         2006-05-11 

In [2]:
#strip non-number and into a type int
#Read the csv we have some None in value,so we replace it to 0
df.Value = df.Value.str.replace('[\D]','', regex = True)
df.Value = pd.to_numeric(df.Value)
df.Value = df.Value.fillna(value = 0)
print(df.Value)

0         200.0 
1         200.0 
2         200.0 
3         200.0 
4         200.0 
          ...   
216925    2000.0
216926    2000.0
216927    2000.0
216928    2000.0
216929    0.0   
Name: Value, Length: 216930, dtype: float64


In [3]:
#Air Date into a date type
df['Air Date'] = df['Air Date'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
print(df['Air Date'])

0        2004-12-31
1        2004-12-31
2        2004-12-31
3        2004-12-31
4        2004-12-31
            ...    
216925   2006-05-11
216926   2006-05-11
216927   2006-05-11
216928   2006-05-11
216929   2006-05-11
Name: Air Date, Length: 216930, dtype: datetime64[ns]


In [4]:
#check duplicated
print(df.duplicated().value_counts())

False    216930
dtype: int64


In [5]:
#a function for search words of the list on the colunms
def search_words(database,colunms,lst):
    index_lst = []
    for i in range(len(database)):
        count = 0
        for j in range(len(lst)):
            if lst[j] in database[colunms][i].lower():
                count +=1
        if count == len(lst):
            index_lst.append(i)
    return pd.DataFrame([database.loc[num] for num in index_lst], columns = database.columns.tolist())

#a new table for 'king' and 'england' all in Question
king_england_in_question = search_words(df,'Question',['king', 'england'])
print(king_england_in_question)

        Show Number   Air Date             Round               Category  \
4953    3003        1997-09-24  Double Jeopardy!  "PH"UN WORDS            
6337    3517        1999-12-14  Double Jeopardy!  Y1K                     
9191    3907        2001-09-04  Double Jeopardy!  WON THE BATTLE          
11710   2903        1997-03-26  Double Jeopardy!  BRITISH MONARCHS        
13454   4726        2005-03-07  Jeopardy!         A NUMBER FROM 1 TO 10   
...      ...               ...        ...                           ...   
208295  4621        2004-10-11  Jeopardy!         THE VIKINGS             
208742  4863        2005-11-02  Double Jeopardy!  BEFORE & AFTER          
213870  5856        2010-02-15  Double Jeopardy!  URANUS                  
216021  1881        1992-11-09  Double Jeopardy!  HISTORIC NAMES          
216789  5070        2006-09-29  Double Jeopardy!  ANCIENT HISTORY         

         Value  \
4953    200.0    
6337    800.0    
9191    800.0    
11710   600.0    
13454   1

In [6]:
#another way to define the function
def search_words2(database, columns, lst):
    filter = lambda x :all(word.lower() in x.lower() for word in lst)
    return database.loc[database[columns].apply(filter)]

king_england_in_question2 = search_words2(df,'Question',['King', 'England'])
print(king_england_in_question2)

        Show Number   Air Date             Round               Category  \
4953    3003        1997-09-24  Double Jeopardy!  "PH"UN WORDS            
6337    3517        1999-12-14  Double Jeopardy!  Y1K                     
9191    3907        2001-09-04  Double Jeopardy!  WON THE BATTLE          
11710   2903        1997-03-26  Double Jeopardy!  BRITISH MONARCHS        
13454   4726        2005-03-07  Jeopardy!         A NUMBER FROM 1 TO 10   
...      ...               ...        ...                           ...   
208295  4621        2004-10-11  Jeopardy!         THE VIKINGS             
208742  4863        2005-11-02  Double Jeopardy!  BEFORE & AFTER          
213870  5856        2010-02-15  Double Jeopardy!  URANUS                  
216021  1881        1992-11-09  Double Jeopardy!  HISTORIC NAMES          
216789  5070        2006-09-29  Double Jeopardy!  ANCIENT HISTORY         

         Value  \
4953    200.0    
6337    800.0    
9191    800.0    
11710   600.0    
13454   1

In [13]:
#Choose the words king and england no matter its capital or little,not inclusive amking or england's
def regex_search(database,columns,lst):
    def check_words(x,lst):
        word_in_x = True
        for word in lst:            
            word = re.compile(r'\b(?i)'+word+r'\b')
            if re.search(word, x) == None:
                word_in_x = False
        return word_in_x               
            
    filter = lambda x: check_words(x,lst)
    return database.loc[database[columns].apply(filter)]

search_table = regex_search(df,'Question',['King', 'england'])
print(search_table)

<ipython-input-13-ec710d8b01bb>:6: DeprecationWarning: Flags not at the start of the expression '\\b(?i)england\\b'
  word = re.compile(r'\b(?i)'+word+r'\b')


        Show Number   Air Date             Round  \
4953    3003        1997-09-24  Double Jeopardy!   
6337    3517        1999-12-14  Double Jeopardy!   
9191    3907        2001-09-04  Double Jeopardy!   
11710   2903        1997-03-26  Double Jeopardy!   
13454   4726        2005-03-07  Jeopardy!          
...      ...               ...        ...          
201168  3515        1999-12-10  Jeopardy!          
204778  5899        2010-04-15  Double Jeopardy!   
208742  4863        2005-11-02  Double Jeopardy!   
213870  5856        2010-02-15  Double Jeopardy!   
216021  1881        1992-11-09  Double Jeopardy!   

                             Category   Value  \
4953    "PH"UN WORDS                   200.0    
6337    Y1K                            800.0    
9191    WON THE BATTLE                 800.0    
11710   BRITISH MONARCHS               600.0    
13454   A NUMBER FROM 1 TO 10          1000.0   
...                       ...             ...   
201168  BEFORE & AFTER          

In [16]:
#calculate the mean of the value
print(df.Value.mean())

#calculate the mean of the value which have king in Question
king_in_question = search_words2(df,'Question',['King'])
print(king_in_question.Value.mean())

739.9884755451067
771.8833850722094


In [17]:
#count the answer of the king in question
print(king_in_question.Answer.value_counts())

Henry VIII        55
Solomon           35
Richard III       33
Louis XIV         31
David             30
                  ..
Dick's ticks      1 
zirconium         1 
casting           1 
\"Camelot\"       1 
making a quilt    1 
Name: Answer, Length: 5268, dtype: int64


In [18]:
#How many questions from the 1990s use the word "Computer" compared to questions from the 2000s
computer_table = search_words2(df,'Question',['Computer'])
computer_table_90s = computer_table[(computer_table['Air Date'] >= datetime(1990,1,1)) \
                                    & (computer_table['Air Date'] <= datetime(1999,12,31))]
computer_table_00s = computer_table[(computer_table['Air Date'] >= datetime(2000,1,1)) \
                                    & (computer_table['Air Date'] <= datetime(2009,12,31))]
print('The question of computer in 90s are:',len(computer_table_90s),',and in 2000s are:',len(computer_table_00s))
print('It is more than ',len(computer_table_00s)-len(computer_table_90s))

The question of computer in 90s are: 98 ,and in 2000s are: 268
It is more than  170
